In [55]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable

In [52]:
v = np.random.normal(0, 1, 3)

In [2]:
bloch_vec = np.array([0.8 ,0, 0])

In [43]:
n_qubits = 2
n_layers = 2
params = np.random.normal(0, np.pi, (n_qubits, n_layers, 3))

def layer(params, j):
    for i in range(n_qubits):
        qml.Rot(params[i, j, 0], params[i, j, 1], params[i, j, 2], wires=i)
    
    #qml.CNOT(wires=[0, 1])

In [44]:
dev = qml.device("default.qubit", wires=3)

In [45]:
Paulis = np.zeros((3, 2, 2), dtype=complex)
Paulis[0] = [[0, 1], [1, 0]]
Paulis[1] = [[0, -1j], [1j, 0]]
Paulis[2] = [[1, 0], [0, -1]]

In [46]:
@qml.qnode(dev)
def circuit(params, A=None):

    # repeatedly apply each layer in the circuit
    #for j in range(n_layers):
    #    layer(params, j)
    layer(params, 0)
    qml.CNOT(wires=[0,1])
    layer(params, 1)

    # returns the expectation of the input matrix A on the first qubit
    return qml.expval(qml.Hermitian(A, wires=0))

In [47]:
# cost function
def cost_fn(params):
    cost = 0
    for k in range(3):
        cost += np.abs(circuit(params, A=Paulis[k]) - bloch_vec[k])

    return cost

In [51]:
# set up the optimizer
opt = qml.AdamOptimizer()

# number of steps in the optimization routine
steps = 200

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((n_qubits, n_layers, 3))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    params = opt.step(cost_fn, params)
    current_cost = cost_fn(params)

    # keeps track of best parameters
    if current_cost < best_cost:
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, current_cost))

Cost after 0 steps is 0.0061
Cost after 10 steps is 0.0901
Cost after 20 steps is 0.0196
Cost after 30 steps is 0.0081
Cost after 40 steps is 0.0076
Cost after 50 steps is 0.0074
Cost after 60 steps is 0.0024
Cost after 70 steps is 0.0033
Cost after 80 steps is 0.0020
Cost after 90 steps is 0.0031
Cost after 100 steps is 0.0064
Cost after 110 steps is 0.0032
Cost after 120 steps is 0.0033
Cost after 130 steps is 0.0044
Cost after 140 steps is 0.0003
Cost after 150 steps is 0.0045
Cost after 160 steps is 0.0069
Cost after 170 steps is 0.0050
Cost after 180 steps is 0.0050
Cost after 190 steps is 0.0041
Cost after 200 steps is 0.0044


In [50]:
# calculate the Bloch vector of the output state
output_bloch_v = np.zeros(3)
for l in range(3):
    output_bloch_v[l] = circuit(best_params, A=Paulis[l])

# print results
print("Target Bloch vector = ", bloch_vec)
print("Output Bloch vector = ", output_bloch_v)

Target Bloch vector =  [0.8 0.  0. ]
Output Bloch vector =  [ 0.80124905  0.00181443 -0.0030508 ]
